# CAPSTONE PROJECT - PART 3
__Michael Gat__  
__General Assembly Santa Monica, Data Science Immersive, Summer 2016__

In this notebook, we'll build upon the model, testing for different numbers of features selected by the Chi2 algorithm.

In [2]:
# IMPORT LIBRARIES ############################################################

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

## READ IN DATA
We have a clean dataset from __Part 1__. We'll import that, then select only the columns we want to deal with at this time. This code is similar to __Part_2__, please review that notebook for additional details/comments.

In [3]:
df = pd.read_csv('diabetic_data_clean.csv')

In [4]:
df_model_1 = df.ix[:,0:15]
df_model_1 = df_model_1.join(df.ix[:,['change', 'diabetesMed', 'age_group', 'readmit']])

In [5]:
df_model_1.dtypes

race                        int64
gender                      int64
admission_type_id           int64
discharge_disposition_id    int64
admission_source_id         int64
time_in_hospital            int64
num_lab_procedures          int64
num_procedures              int64
num_medications             int64
number_outpatient           int64
number_emergency            int64
number_inpatient            int64
number_diagnoses            int64
max_glu_serum               int64
A1Cresult                   int64
change                      int64
diabetesMed                 int64
age_group                   int64
readmit                     int64
dtype: object

## BUILD MORE SOPHISTICATED FEATURE SELECTION
Still working with the recommended Chi2, test for different numbers of features, using code adapted from __Part2__.
Key results in Capstone_Results.xlsx

In [6]:
X = df_model_1.ix[:,0:17]
y = df_model_1.ix[:,18]
#X_norm = StandardScaler().fit_transform(X)

X_std = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.75)

classifiers = [DecisionTreeClassifier(max_depth=7), \
RandomForestClassifier(max_depth=7, n_estimators=10, max_features=1), \
GaussianNB(), LogisticRegression()]

for numtest in range (3,10):
    ch2 = SelectKBest(chi2, k=numtest)
    X_train_fit = ch2.fit_transform(X_train, y_train)
    print 'SELECTED FEATURES:' + str(numtest)
    col_indices = ch2.get_support(indices=True)
    for i in col_indices:
        print X_train.columns.values[i]
    print ch2.scores_
    print
    X_test_xform = ch2.transform(X_test)
    
    for clf in classifiers:
        clf.fit(X_train_fit, y_train)
        score = clf.score(X_test_xform, y_test)
        y_pred = clf.predict(X_test_xform)
        y_pred = clf.predict(X_test_xform)
        cm = confusion_matrix(y_test, y_pred)
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        print clf
        print score
        print cm
        print "AUC Metrics:"
        print auc(fpr, tpr)
        print

    
    

SELECTED FEATURES:3
discharge_disposition_id
number_emergency
number_inpatient
[  1.73968535e-01   4.19195112e-01   9.42000153e+00   4.66594545e+02
   2.66501730e+00   9.10867309e+01   5.19059937e+01   2.38182811e+01
   1.64531678e+02   5.12599264e+01   4.35274663e+02   2.35338511e+03
   5.32879553e+01   8.48713414e-01   5.50590197e+00   5.84219056e+00
   2.34749296e+00]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
0.887323943662
[[67604   187]
 [ 8413   121]]
AUC Metrics:
0.505710050767

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=1, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_sco